In [1]:
import pandas as pd
import numpy as np
from scipy import stats

#load dataset
df = pd.read_csv('training_set_VU_DM.csv')

In [2]:
# convert date to numerical feature 
df = df.sort_values(by="date_time")
df["date_time"] = (pd.to_datetime(df["date_time"]) - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")

In [3]:
# drop columns with lot of nans and uncorrelated columns
drop_columns_nans = ['visitor_hist_starrating', 'visitor_hist_adr_usd','srch_query_affinity_score','comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate']
drop_columns_low_correlation = ['site_id','prop_log_historical_price', 'srch_destination_id']
df = df.drop(drop_columns_nans, axis=1)
df = df.drop(drop_columns_low_correlation, axis=1)

In [4]:
# Function to remove outliers based on z-scores
def remove_outliers(group):
    # Calculate the absolute z-scores for the 'price_usd' column within the group
    z_scores = np.abs(stats.zscore(group['price_usd']))
    # Return the rows where the z-score is less than or equal to 1 (keeping non-outliers)
    return group[(z_scores <= 1)]

# Apply the outlier removal function to each group of 'prop_id'
df2 = df.copy(deep=True)
df_wo_outliers = df2.groupby('prop_id').apply(remove_outliers).reset_index(drop=True)

# Calculate the mean 'price_usd' for each 'prop_id' after removing outliers
hotel_means = df_wo_outliers.groupby('prop_id')['price_usd'].mean().reset_index()

# Merge the original dataframe with the calculated means on 'prop_id'
df_merged = df.merge(hotel_means, on='prop_id', suffixes=('', '_mean'))

In [ ]:
# Make a

In [9]:
# Define a threshold for price adjustment
too_low = 0.3
too_high = 2

# Function to replace prices significantly lower than the mean with the mean price
def replace_with_mean(row):
    # If the 'price_usd' is less than 30% of the 'price_usd_mean', replace it with the mean
    if row['price_usd'] < row['price_usd_mean'] * too_low or row['price_usd'] > row['price_usd_mean'] * too_high:
        return row['price_usd_mean']
    else:
        return row['price_usd']

# Apply the price adjustment function to each row
df_merged['adjusted_price'] = df_merged.apply(replace_with_mean, axis=1)

# Drop the original 'price_usd' column and rename 'adjusted_price' to 'price_usd'
df_merged = df_merged.drop('price_usd', axis=1).rename(columns={'adjusted_price': 'price_usd'})

In [6]:
df_merged.to_csv('cleaned.csv')

In [ ]:
df

,srch_id,date_time,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
1219448,81578,1351728509,219,219,93974,4,4.5,1,5.17,0.6352,...,0,1,1,355.82,1,0.0,NaN,0,NaN,0
1219451,81578,1351728509,219,219,133689,4,4.5,1,5.19,0.6166,...,0,1,1,355.78,1,0.0,NaN,0,NaN,0
1219450,81578,1351728509,219,219,119527,4,4.5,1,5.18,0.6132,...,0,1,1,355.79,1,0.0,NaN,0,NaN,0
1219449,81578,1351728509,219,219,95095,5,4.5,1,4.93,0.1350,...,0,1,1,355.70,1,0.0,NaN,0,NaN,0
1219419,81578,1351728509,219,219,4080,4,4.5,1,5.18,0.3795,...,0,1,1,356.05,1,0.0,100.0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003762,67044,1372636704,13,219,68491,3,4.0,0,3.87,0.6124,...,0,1,1,NaN,1,0.0,NaN,0,NaN,0
1003761,67044,1372636704,13,219,59546,4,4.0,0,3.83,0.6580,...,0,1,1,NaN,1,NaN,NaN,0,NaN,0
1003760,67044,1372636704,13,219,46809,4,4.0,0,3.78,0.6438,...,0,1,1,NaN,1,0.0,11.0,0,NaN,0
1003759,67044,1372636704,13,219,39164,3,4.0,0,3.71,0.1973,...,0,1,1,NaN,1,0.0,NaN,0,NaN,0


In [ ]:
df_merged

,srch_id,date_time,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,srch_saturday_night_bool,orig_destination_distance,random_bool,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,price_usd_mean,price_usd
0,81578,1351728509,219,219,93974,4,4.5,1,5.17,0.6352,...,1,355.82,1,0.0,NaN,0,NaN,0,215.896672,195.00
1,59740,1351762001,219,219,93974,4,4.5,1,5.17,0.3128,...,0,1479.63,0,0.0,NaN,0,NaN,0,215.896672,143.00
2,200035,1351769274,219,219,93974,4,4.5,1,5.17,0.3128,...,1,382.84,0,0.0,24.0,0,NaN,0,215.896672,167.00
3,310536,1351794345,219,219,93974,4,4.5,1,5.17,0.3128,...,0,2391.65,0,0.0,NaN,0,NaN,0,215.896672,159.00
4,168554,1351945381,219,219,93974,4,4.5,1,5.17,0.3128,...,0,658.98,0,0.0,7.0,0,NaN,0,215.896672,289.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4940975,181169,1372630349,31,31,76724,0,NaN,0,1.61,NaN,...,0,185.10,1,NaN,NaN,0,NaN,0,365.550000,365.55
4940976,49471,1372589704,219,219,30540,4,0.0,0,0.00,NaN,...,0,NaN,0,NaN,NaN,0,NaN,0,144.125000,139.00
4940977,174004,1372622208,100,219,30540,4,0.0,0,0.00,NaN,...,0,270.95,0,NaN,NaN,0,NaN,0,144.125000,149.25
4940978,259120,1372620379,129,152,84169,1,0.0,0,0.00,NaN,...,0,NaN,1,NaN,NaN,0,NaN,0,89.760000,59.84
